# Recommendation system:

    this project is done by: 
        Youssef Abid
        Nada Zaddem
        Nadhem Zmander

    Under the supervision of Mrs. Tbarki khaoula

## Neural Collaborative Filtering

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse import vstack
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout
from keras.layers.merge import Dot, multiply, concatenate
from keras.utils import np_utils
from keras.layers import merge
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from sklearn.metrics import mean_absolute_error

%matplotlib inline

Using TensorFlow backend.


In [3]:
events = pd.read_csv('./input/events.csv')

In [4]:
events = events.sort_values(by=['visitorid', 'timestamp'])
events.head()

,timestamp,visitorid,event,itemid,transactionid
1361687,1442004589439,0,view,285930,NaN
1367212,1442004759591,0,view,357564,NaN
1367342,1442004917175,0,view,67045,NaN
830385,1439487966444,1,view,72028,NaN
742616,1438969904567,2,view,325215,NaN


# Filtring data

Assign a unique number between 0 and # users to each user. the same for items.
convert actions to weights.

In [5]:
user_item = dict()
action_weights = [2,3,5]

visitorid_to_index_mapping  = {}
itemid_to_index_mapping  = {}
vid = 0
iid = 0

buyers = [] # we can take only users that made a purchase but that will reduced very much our data set

for index, row in enumerate(events.itertuples()):
    if row.visitorid not in visitorid_to_index_mapping:
        visitorid_to_index_mapping[row.visitorid] = vid
        vid += 1
        
    if  row.itemid not in itemid_to_index_mapping:
        itemid_to_index_mapping[row.itemid] = iid
        iid += 1
    
    visitor_index = visitorid_to_index_mapping[row.visitorid]
    item_index = itemid_to_index_mapping[row.itemid]
    
    user_item[index] = {'visitorId':visitor_index, 'itemId':item_index, 'value':0, 'timestamp':row.timestamp}
    if row.event == 'view':
        user_item[index]['value'] = max(user_item[index]['value'], action_weights[0]) 
    elif row.event == 'addtocart':
        user_item[index]['value'] = max(user_item[index]['value'], action_weights[1])
    elif row.event == 'transaction':
        user_item[index]['value'] = max(user_item[index]['value'], action_weights[2])

data = pd.DataFrame(user_item)

In [22]:
data = data.transpose()
data = data.sort_values(by=['visitorId', 'timestamp'])
data.head(10)

,itemId,timestamp,value,visitorId
0,0,1442004589439,2,0
1,1,1442004759591,2,0
2,2,1442004917175,2,0
3,3,1439487966444,2,1
4,4,1438969904567,2,2
5,4,1438970013790,2,2
6,5,1438970212664,2,2
7,6,1438970468920,2,2
8,7,1438970905669,2,2
9,7,1438971444375,2,2


In [23]:
print('Number of unique visitors ' + str(len(data.visitorId.unique())))
print('Number of unique items ' + str(len(data.itemId.unique())))

Number of unique visitors 1407580
Number of unique items 235061


Split the data set into train and test set.

In [27]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.998)

### Model 1: simple linear model 

The first model is a simple linear model where we learn a dense representation of each items and visitors in our data set.

In [28]:
dim_embedddings = 30
num_items = len(data.itemId.unique())
num_visitors = len(data.visitorId.unique())

In [29]:
m_inputs = Input(shape=(1,), dtype='int32')
m = Embedding(num_items + 1, dim_embedddings, name="item")(m_inputs)

u_inputs = Input(shape=(1,), dtype='int32')
u = Embedding(num_visitors + 1, dim_embedddings, name="visitor")(u_inputs)

o = multiply([m, u])
o = Dropout(0.5)(o)
o = Flatten()(o)
o = Dense(1)(o)

rec_model = Model(inputs=[m_inputs, u_inputs], outputs=o)
rec_model.compile(loss='mae', optimizer='adam', metrics=["mae"])

In [30]:
history = rec_model.fit([train.itemId, train.visitorId], train.value, epochs=5, verbose=2, validation_split=0.1)

Train on 4960 samples, validate on 552 samples
Epoch 1/5
85s - loss: 1.9765 - mean_absolute_error: 1.9765 - val_loss: 1.8952 - val_mean_absolute_error: 1.8952
Epoch 2/5
82s - loss: 1.8190 - mean_absolute_error: 1.8190 - val_loss: 1.7399 - val_mean_absolute_error: 1.7399
Epoch 3/5
84s - loss: 1.6576 - mean_absolute_error: 1.6576 - val_loss: 1.5847 - val_mean_absolute_error: 1.5847
Epoch 4/5
85s - loss: 1.4870 - mean_absolute_error: 1.4870 - val_loss: 1.4295 - val_mean_absolute_error: 1.4295
Epoch 5/5
83s - loss: 1.3011 - mean_absolute_error: 1.3011 - val_loss: 1.2742 - val_mean_absolute_error: 1.2742


In [31]:
print(mean_absolute_error(test.value, rec_model.predict([test.itemId, test.visitorId])))

0.70621256481148


### Model 2:  With bias

In this model we introduce a bias. The first model does not explicitly take into account the bias that a user might buy all items he viewed or a item having consistently viewed but not bought by all users.

In [48]:
bias = 1
m_inputs = Input(shape=(1,), dtype='int32')
m = Embedding(num_items + 1, dim_embedddings, name="item")(m_inputs)
m_bias = Embedding(num_items + 1, bias, name="itembias")(m_inputs)

u_inputs = Input(shape=(1,), dtype='int32')
u = Embedding(num_visitors + 1, dim_embedddings, name="visitor")(u_inputs)
u_bias = Embedding(num_visitors + 1, bias, name="visitorbias")(u_inputs)

o = multiply([m, u])
o = concatenate([o, m_bias, u_bias])
o = Dropout(0.5)(o)
o = Flatten()(o)
o = Dense(1)(o)

rec_model = Model(inputs=[m_inputs, u_inputs], outputs=o)
rec_model.compile(loss='mae', optimizer='adam', metrics=["mae"])

In [49]:
history = rec_model.fit([train.itemId, train.visitorId], train.value, epochs=5, verbose=2, validation_split=0.1)

Train on 2480 samples, validate on 276 samples
Epoch 1/5
104s - loss: 2.0078 - mean_absolute_error: 2.0078 - val_loss: 2.0083 - val_mean_absolute_error: 2.0083
Epoch 2/5
51s - loss: 1.9269 - mean_absolute_error: 1.9269 - val_loss: 1.9299 - val_mean_absolute_error: 1.9299
Epoch 3/5
51s - loss: 1.8425 - mean_absolute_error: 1.8425 - val_loss: 1.8512 - val_mean_absolute_error: 1.8512
Epoch 4/5
50s - loss: 1.7528 - mean_absolute_error: 1.7528 - val_loss: 1.7720 - val_mean_absolute_error: 1.7720
Epoch 5/5
51s - loss: 1.6548 - mean_absolute_error: 1.6548 - val_loss: 1.6926 - val_mean_absolute_error: 1.6926


In [16]:
print(mean_absolute_error(test.value, rec_model.predict([test.itemId, test.visitorId])))

0.6893623254451426


### Model 3: Neural collaborative filtering

In [50]:
mf_dim = 8
layers = [64, 32, 16, 8]
visitor_input = Input(shape=(1,), dtype='int32', name = 'visitor_input')
item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

MF_Embedding_Visitor = Embedding(input_dim = num_visitors + 1, output_dim = mf_dim, name = 'mf_embedding_visitor', input_length=1)
MF_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = mf_dim, name = 'mf_embedding_item', input_length=1)

MLP_Embedding_Visitor = Embedding(input_dim = num_visitors + 1, output_dim = int(layers[0]/2), name = "mlp_embedding_visitor", input_length=1)
MLP_Embedding_Item = Embedding(input_dim = num_items + 1, output_dim = int(layers[0]/2), name = 'mlp_embedding_item', input_length=1)

mf_visitor_latent = Flatten()(MF_Embedding_Visitor(visitor_input))
mf_item_latent = Flatten()(MF_Embedding_Item(item_input))

mf_vector = merge([mf_visitor_latent, mf_item_latent], mode = 'mul')

mlp_visitor_latent = Flatten()(MLP_Embedding_Visitor(visitor_input)) 
mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))

mlp_vector = merge([mlp_visitor_latent, mlp_item_latent], mode = 'concat')

for idx in range(1, len(layers)):
    layer = Dense(layers[idx], activation='relu', name="layer%d" %idx)
    mlp_vector = layer(mlp_vector)
                  
predict_vector = merge([mf_vector, mlp_vector], mode = 'concat')
prediction = Dense(1, activation='sigmoid', init='lecun_uniform', name = "prediction")(predict_vector)

model = Model(input=[visitor_input, item_input], output=prediction)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

In [51]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
visitor_input (InputLayer)       (None, 1)             0                                            
____________________________________________________________________________________________________
item_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
mlp_embedding_visitor (Embedding (None, 1, 32)         45042592    visitor_input[0][0]              
____________________________________________________________________________________________________
mlp_embedding_item (Embedding)   (None, 1, 32)         7521984     item_input[0][0]                 
___________________________________________________________________________________________

In [22]:
history = model.fit([train.visitorId, train.itemId], train.value, epochs=1, verbose=2, validation_split=0.1)

Train on 72601 samples, validate on 8067 samples
Epoch 1/1
11s - loss: -3.9917e+01 - val_loss: -3.9782e+01


In [20]:
print(mean_absolute_error(test.rating, model.predict([test.visitorId, test.itemId])))

2.5652217389130545


## References

* [Neural Collaborative Filtering](https://arxiv.org/abs/1708.05031)